In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import itertools
import random
import os

In [2]:
!git clone https://github.com/joneikholmkea/veggies.git

fatal: destination path 'veggies' already exists and is not an empty directory.


In [2]:
trainingFiles = "veggies/trainRed"
testFiles = "veggies/testRed"
validationFiles = 'veggies/validation'
targetSize=[100,100]

In [6]:
image_files = glob(trainingFiles + '/*/*.jp*g') # load alle billeder som slutter med jpg eller jpeg
folders = glob(trainingFiles + '/*') # giver alle undermapper

In [7]:
resNet= ResNet50(input_shape=targetSize + [3], weights='imagenet', include_top=False)

In [8]:
for layer in resNet.layers: #kun vores del af modellen skal trænes
  layer.trainable = False

In [9]:
# Add dropout to the model
layers = Flatten()(resNet.output)
layers = Dense(30, activation='relu')(layers)
layers = Dropout(0.2)(layers) # with 20% probability drop a neuron
layers = Dense(len(folders), activation='softmax')(layers)

In [10]:
model = Model(inputs=resNet.input, outputs=layers)
model.compile(
    optimizer = 'adam',
    metrics=['accuracy'],
    loss='sparse_categorical_crossentropy'
)

In [11]:
# Create an instance of the ImageDataGenerator class
dataGen = ImageDataGenerator(
    rotation_range = 20,  # Randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # Randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # Randomly shift images vertically (fraction of total height)
    shear_range=0.1,  # Set range for random shear
    horizontal_flip=True,  # Randomly flip inputs horizontally
    vertical_flip=True,  # Randomly flip inputs vertically
    preprocessing_function = preprocess_input  # Function applied to each input image
)

In [12]:
# Use the ImageDataGenerator instance to read images from the directory and preprocess them
training_set = dataGen.flow_from_directory(
    trainingFiles,  # Path to the directory
    target_size = targetSize,  # The dimensions to which all images found will be resized
    shuffle=True,  # Whether to shuffle the data
    batch_size=50,  # Number of images to process at a time
    class_mode='sparse'  # Mode for yielding the targets: 'sparse' means that the categorical labels will be returned as 1D integer labels
)

Found 3000 images belonging to 15 classes.


In [13]:
test_set = dataGen.flow_from_directory(
    testFiles,
    target_size = targetSize,
    shuffle=False,
    batch_size=50,
    class_mode='sparse' # Laver One-hot om til tal
)

Found 3000 images belonging to 15 classes.


In [14]:
validation_Set = dataGen.flow_from_directory(
    validationFiles,
    target_size = targetSize,
    shuffle=False,
    batch_size=50,
    class_mode='sparse' # Laver One-hot om til tal
)

Found 3000 images belonging to 15 classes.


In [22]:
model.fit(
    training_set,
    validation_data=test_set, #bruger test_set som guide til at burdere om vægte justeres optimalt
    epochs=10
)
model.save('my_model.keras')

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 41s 669ms/step - accuracy: 0.7236 - loss: 0.8419 - val_accuracy: 0.8780 - val_loss: 0.4435
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 44s 724ms/step - accuracy: 0.7694 - loss: 0.7478 - val_accuracy: 0.9033 - val_loss: 0.3675
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 42s 702ms/step - accuracy: 0.8112 - loss: 0.5780 - val_accuracy: 0.9143 - val_loss: 0.3047
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 41s 681ms/step - accuracy: 0.8207 - loss: 0.5722 - val_accuracy: 0.9260 - val_loss: 0.2725
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 43s 715ms/step - accuracy: 0.8174 - loss: 0.5577 - val_accuracy: 0.9173 - val_loss: 0.3212
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 41s 689ms/step - accuracy: 0.8410 - loss: 0.5234 - val_accuracy: 0.9267 - val_loss: 0.2595
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 41s 682ms/step - accuracy: 0.8490 - loss: 0.4747 - val_accuracy: 0.9353 - val_loss: 0.2689
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 41s 672ms/step - accuracy: 0.8584 - loss: 0.4252 - val_accu

In [37]:
subfolders = [f.path for f in os.scandir(trainingFiles) if f.is_dir()]

# Choose a random subfolder
random_subfolder = random.choice(subfolders)

# Get a list of all images in the chosen subfolder
images = [f for f in os.listdir(random_subfolder) if os.path.isfile(os.path.join(random_subfolder, f))]

# Choose a random image
random_image = random.choice(images)
# Construct the full path to the random image
image_path = os.path.join(random_subfolder, random_image)

print("Chosen image path:", image_path)

# Load the image
img = image.load_img(image_path, target_size=(100, 100))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Expand dimensions to fit the model's expected input shape
img_array = np.expand_dims(img_array, axis=0)

# Make a prediction
result = model.predict(img_array)

# Get the index of the highest probability
predicted_class_index = np.argmax(result)

class_names = list(training_set.class_indices.keys())
predicted_class_name = class_names[predicted_class_index]
# Print the predicted class name
print("Predicted Class Name:", predicted_class_name, "With probability:", result[0][predicted_class_index])

Chosen image path: veggies/trainRed/Potato/0376.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted Class Name: Potato With probability: 0.99197257
